In [1]:
import os
os.chdir('/kaggle/input/mobilefacenet123/pytorch/default/1/Face_Recognition_using_pytorch-master')

In [2]:
!pip install torchsummary

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from ArcFace.mobile_model import mobileFaceNet, Arcloss
from tqdm import tqdm
import os
from torchsummary import summary



In [13]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Hyperparameters
num_epochs = 30
batch_size = 16
learning_rate = 0.001
num_classes = 100

Using device: cuda


In [5]:
# Data augmentation and preprocessing
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(5),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

val_transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [6]:
# Load your dataset
train_dataset = datasets.ImageFolder(root='/kaggle/input/dataset123/train', transform=train_transform)
val_dataset = datasets.ImageFolder(root='/kaggle/input/dataset123/val', transform=val_transform)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")

Train dataset size: 1947
Validation dataset size: 417


In [7]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

In [8]:
# Load the pre-trained model
model = mobileFaceNet()
checkpoint_path = '/kaggle/input/mobilefacenet123/pytorch/default/1/Face_Recognition_using_pytorch-master/ArcFace/model/pretrained.pth'
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))

# Inspect checkpoint keys
print(f"Checkpoint keys: {checkpoint.keys()}")

# Load model state
model.load_state_dict(checkpoint['backbone_net_list'])
model = model.to(device)

Checkpoint keys: dict_keys(['epoch', 'backbone_net_list', 'arcloss_net_list'])


In [9]:
# Freeze early layers
for param in model.parameters():
    param.requires_grad = False

# Unfreeze the last few layers
for param in model.conv2.parameters():
    param.requires_grad = True
for param in model.linear7.parameters():
    param.requires_grad = True
for param in model.linear1.parameters():
    param.requires_grad = True

# Loss functions
criterion_ce = nn.CrossEntropyLoss().to(device)
criterion = Arcloss(num_classes, s=30, m=0.5).to(device)

# Optimizer and scheduler
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [10]:
# Training function
def train(model, train_loader, arcloss, criterion_ce, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (images, labels) in enumerate(tqdm(train_loader)):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)

        # Debugging: Check output and label shapes
        print(f"Batch {batch_idx}")
        print(f"Outputs shape: {outputs.shape}")
        print(f"Labels shape: {labels.shape}")
        print(f"Labels max: {labels.max()}, Labels min: {labels.min()}")

        assert labels.max() < num_classes, f"Label {labels.max()} is out of range."

        arc_outputs = arcloss(outputs, labels)
        loss = criterion_ce(arc_outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = arc_outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        if batch_idx % 10 == 0:
            print(f"Batch {batch_idx}, Loss: {loss.item():.4f}")

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

In [11]:
# Validation function
def validate(model, val_loader, arcloss, criterion_ce, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(tqdm(val_loader)):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)

            # Debugging: Check output and label shapes
            print(f"Batch {batch_idx}")
            print(f"Outputs shape: {outputs.shape}")
            print(f"Labels shape: {labels.shape}")
            print(f"Labels max: {labels.max()}, Labels min: {labels.min()}")

            assert labels.max() < num_classes, f"Label {labels.max()} is out of range."

            arc_outputs = arcloss(outputs, labels)
            loss = criterion_ce(arc_outputs, labels)

            running_loss += loss.item()
            _, predicted = arc_outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / len(val_loader)
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

In [14]:
# Training loop
best_val_loss = float('inf')
patience = 5
epochs_no_improve = 0

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")

    train_loss, train_acc = train(model, train_loader, criterion, criterion_ce, optimizer, device)
    val_loss, val_acc = validate(model, val_loader, criterion, criterion_ce, device)

    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), '/kaggle/working/finetuned.pth')
        print("Saved best model!")
    else:
        epochs_no_improve += 1

    if epochs_no_improve == patience:
        print('Early stopping!')
        break

    scheduler.step()

print("Training completed!")



Epoch 1/30


  3%|▎         | 4/122 [00:00<00:11, 10.22it/s]

Batch 0
Outputs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Labels max: 96, Labels min: 3
Batch 0, Loss: 0.4093
Batch 1
Outputs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Labels max: 96, Labels min: 1
Batch 2
Outputs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Labels max: 99, Labels min: 12
Batch 3
Outputs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Labels max: 91, Labels min: 2
Batch 4
Outputs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Labels max: 97, Labels min: 3
Batch 5
Outputs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Labels max: 94, Labels min: 7


  9%|▉         | 11/122 [00:00<00:06, 18.30it/s]

Batch 6
Outputs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Labels max: 99, Labels min: 3
Batch 7
Outputs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Labels max: 86, Labels min: 4
Batch 8
Outputs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Labels max: 99, Labels min: 6
Batch 9
Outputs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Labels max: 99, Labels min: 3
Batch 10
Outputs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Labels max: 94, Labels min: 3
Batch 10, Loss: 0.1447
Batch 11
Outputs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Labels max: 99, Labels min: 7


...

Batch 4
Outputs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Labels max: 23, Labels min: 18
Batch 5
Outputs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Labels max: 28, Labels min: 23
Batch 6
Outputs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Labels max: 32, Labels min: 28
Batch 7
Outputs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Labels max: 37, Labels min: 32
Batch 8
Outputs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Labels max: 42, Labels min: 38
Batch 9
Outputs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Labels max: 46, Labels min: 42
Batch 10
Outputs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Labels max: 51, Labels min: 46
Batch 11
Outputs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Labels max: 56, Labels min: 52


 70%|███████   | 19/27 [00:00<00:00, 33.14it/s]

Batch 12
Outputs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Labels max: 59, Labels min: 56
Batch 13
Outputs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Labels max: 62, Labels min: 59
Batch 14
Outputs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Labels max: 64, Labels min: 62
Batch 15
Outputs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Labels max: 67, Labels min: 65
Batch 16
Outputs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Labels max: 70, Labels min: 67
Batch 17
Outputs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Labels max: 73, Labels min: 71
Batch 18
Outputs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Labels max: 76, Labels min: 74
Batch 19
Outputs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Labels max: 79, Labels min: 76
Batch 20
Outputs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Labels max: 82, Labels min: 80
Batch 21
Outputs shape: torc

100%|██████████| 27/27 [00:00<00:00, 27.81it/s]


Batch 22
Outputs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Labels max: 89, Labels min: 85
Batch 23
Outputs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Labels max: 93, Labels min: 89
Batch 24
Outputs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Labels max: 96, Labels min: 93
Batch 25
Outputs shape: torch.Size([16, 128])
Labels shape: torch.Size([16])
Labels max: 99, Labels min: 96
Batch 26
Outputs shape: torch.Size([1, 128])
Labels shape: torch.Size([1])
Labels max: 99, Labels min: 99
Train Loss: 0.1240, Train Acc: 0.9759
Val Loss: 3.1005, Val Acc: 0.6667
Saved best model!
Training completed!
